In [1]:
# searches for a tile path given its location
# note: requires unix for system 'cat' command.

import numpy as np
import subprocess
import re

In [2]:
# load the coefficient paths from pgp data and generate tile path, step, and phase.
coefPaths = np.load("./tiling/hiq-pgp-info")
tile_path = np.trunc(coefPaths/(16**5))
tile_step = np.trunc((coefPaths - tile_path*16**5)/2)
tile_phase = np.trunc((coefPaths - tile_path*16**5 - 2*tile_step))
vhex = np.vectorize(hex)
vectorizedPath = vhex(tile_path.astype('int'))
vectorizedStep = vhex(tile_step.astype('int'))

In [3]:
# search for a tile
def tileSearch(arg):
    vecpath = str(vectorizedPath[int(arg)])
    vecpath = vecpath[2:].zfill(4)
    proc = subprocess.check_output("cat ./tiling/assembly.00.hg19.fw.fwi | grep :" + vecpath, shell=True)
    return proc

In [4]:
# get the location of a tile
def getTileLocation(raw_tile_data):
    split_raw = raw_tile_data.split('\t')
    begin = int(split_raw[2])
    sequence = int(split_raw[1])
    hexVal = split_raw[0].split(':')[2]
    cmdToRun = "bgzip -c -b %d -s %d -d ./tiling/assembly.00.hg19.fw.gz | grep -B1 \"%s\s\"" % (begin, sequence, hexVal)
    return subprocess.check_output(cmdToRun, shell=True)

In [7]:
# load generated coefficients (sorted by weight)
coefs = np.load("coefs.pkl")

In [8]:
print(coefs)

[ 200879 2364942  688238 2013689 1704500 1704512 1704504 1704532  796640
 2151203 1138855  344935 1704538]


In [9]:
# search for the specific tile location from the coefficients
tileLocations = []
for item in coefs:
    tile = tileSearch(item)
    print(tile, item, '\n')
    tileLocations.append(tile)

CalledProcessError: Command 'cat ./tiling/assembly.00.hg19.fw.fwi | grep :003e' returned non-zero exit status 255.

In [28]:
# get the location of the tile with the highest coefficient
print(getTileLocation(tileLocations[0]))

IndexError: list index out of range